In [6]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
import tensorflow.keras.datasets as datasets
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

In [4]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [7]:
inp = keras.Input(shape = (28, 28, 1))
conv1 = layers.Conv2D(64, 3, padding = 'same')(inp)
relu1 = layers.ReLU()(conv1)
conv2 = layers.Conv2D(128, 3, padding = 'same')(relu1)
relu2 = layers.ReLU()(conv2)
flatten = layers.Flatten()(relu2)
dense1 = layers.Dense(10)(flatten)

model = Model(inputs = [inp, ], outputs = [dense1, ])
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 re_lu (ReLU)                (None, 28, 28, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 re_lu_1 (ReLU)              (None, 28, 28, 128)       0         
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 10)                100353

In [28]:
acc_metric = keras.metrics.SparseCategoricalAccuracy(name = "Accuracy")

class CustomFit(keras.Model):
    def __init__(self, model):
        super(CustomFit, self).__init__()
        self.model = model

    def compile(self, optimizer, loss):
        super(CustomFit, self).compile()
        self.loss = loss
        self.optimizer = optimizer

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self.model(x, training = True)
            # loss = self.compiled_loss(y, y_pred)
            loss = self.loss(y, y_pred)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # self.compiled_metrics.update_state(y, y_pred)
        acc_metric.update_state(y, y_pred)

        return {"loss": loss, "accuracy": acc_metric.result()}
        # return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        x, y = data

        y_pred = self.model(x, training = False)
        # loss  = self.compiled_loss(y, y_pred)
        loss = self.loss(y, y_pred)
        acc_metric.update_state(y, y_pred)

        return {"loss": loss, "accuracy": acc_metric.result()}

In [31]:
# model.fit(x_train, y_train, epochs = 10, verbose = 2, batch_size = 64)
# model.evaluate(x_test, y_test, verbose = 2, batch_size = 2)

history = CustomFit(model)
history.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    # metrics = ['accuracy'],
)
history.fit(x_train, y_train, epochs = 2, batch_size = 32)
history.evaluate(x_test, y_test, batch_size = 32, verbose = 2)

Epoch 1/2
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0225 - accuracy: 0.9924
Epoch 2/2
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0112 - accuracy: 0.9936
313/313 - 1s - loss: 1.8812e-04 - accuracy: 0.9938 - 1s/epoch - 4ms/step


[0.9937752485275269, 0.00018812227062880993]